In [1]:
import os
import csv
import json
import pandas as pd
from collections import Counter, defaultdict

It's doing better than before on shapes.. and it can now build diamonds sometimes.. When looking at the visualizer, I saw it messing up square as rectangle quite a bit. Also, for diamonds the size was always 3 for the examples I saw. Can you check with the visualizer that when we detect a square it's actually a square and for diamonds it's making of varied sizes? Also, we need the orientation breakdown.

In [2]:
home=%pwd
#Nebula
# csv_path = home + '/llama_synth_level_one_output.csv'
#Nebula plus finetuning one
# csv_path = home + '/llama_aug_synth_level_one_output_for_analysis.csv'
#Nebula plus finetuning two
# csv_path = home + '/llama_aug_synth_level_one_output_for_analysis_v2.csv'
#Neural builder level one
csv_path = home + '/neural_builder_lvl1_synth_output_for_analysis.csv'

In [ ]:
# with open(csv_path, newline='') as f:
#     reader = csv.reader(f)
#     data = list(reader)

In [3]:
df = pd.read_csv(csv_path)

In [4]:
df.shape

(1368, 8)

In [5]:
df.head(10)

,Shape,Color,Shape_corr,Size_corr,Loc_corr,Orient_corr,In_bounds,Full_struct_corr
0,diagonal,1,0,0,1.0,NaN,1,0
1,row,1,1,1,NaN,NaN,1,1
2,square,1,0,0,1.0,0.0,1,0
3,tower,1,0,0,0.0,NaN,1,0
4,rectangle_botched,0,0,0,0.0,0.0,1,0
5,diamond,1,0,0,NaN,1.0,1,0
6,row,1,1,0,1.0,NaN,1,0
7,tower_botched,0,0,0,0.0,0.0,1,0
8,square,1,0,0,0.0,NaN,1,0
9,tower,1,0,0,1.0,NaN,1,0


In [ ]:
Counter(df['Shape_corr'].tolist())

In [ ]:
Counter(df['Size_corr'].tolist())

In [ ]:
Counter(df['Color'].tolist())

In [ ]:
Counter(df['Full_struct_corr'].tolist())

In [ ]:
#Counter(df['Loc_corr'].tolist())

In [ ]:
#which samples didn't get color right? Only when the moves were botched
df.loc[df['Color'] == 0]

In [6]:
#what the breakdown of correct and incorrect shapes?
#Correct:
Counter(df.loc[df['Shape_corr'] == 1, 'Shape'].tolist())

Counter({'tower': 381, 'row': 164})

In [7]:
#what the breakdown of correct and incorrect shapes?
#INorrect:
Counter(df.loc[df['Shape_corr'] == 0, 'Shape'].tolist())

Counter({'square': 186,
         'diagonal': 163,
         'diamond': 134,
         'rectangle': 108,
         'tower': 75,
         'tower_botched': 48,
         'rectangle_botched': 36,
         'square_botched': 30,
         'cube': 22,
         'diamond_botched': 10,
         'diagonal_botched': 5,
         'row_botched': 4,
         'cube_botched': 2})

In [ ]:
#Incorrect:
Counter(df.loc[df['Shape_corr'].isin([0,1]), 'Shape'].tolist())

In [ ]:
#what the breakdown of correct and incorrect sizes?
#Correct:
Counter(df.loc[df['Size_corr'] == 1, 'Shape'].tolist())

In [ ]:
#what the breakdown of correct and incorrect locations?
#Correct:
Counter(df.loc[df['Loc_corr'] == 1, 'Shape'].tolist())

In [ ]:
Counter(df[(df['Loc_corr'].isin([1,0]))]['Shape'].tolist())

In [ ]:
Counter(df[(df['Size_corr'] == 0)&(df['Shape_corr'] == 1)]['Shape'].tolist())

In [8]:
Counter(df[(df['Loc_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'tower': 303, 'row': 126})

In [9]:
Counter(df[(df['Loc_corr'] == 1)&(df['Shape_corr'] == 1)]['Shape'].tolist())

Counter({'row': 41})

In [ ]:
#Shape corr and location specified
Counter(df[(df['Shape_corr'] == 1)&(df['Loc_corr'].isin([1]))]['Shape'].tolist())

In [ ]:
#Shape corr and is in bounds
Counter(df[(df['Shape_corr'] == 1)&(df['In_bounds'] == 1)&(df['Loc_corr'].isin([0, 1]))]['Shape'].tolist())

In [ ]:
#Shape corr and is in bounds
Counter(df[(df['Shape_corr'] == 1)&(df['In_bounds'] == 1)&(df['Loc_corr'].isin([1]))]['Shape'].tolist())

In [ ]:
Counter(df[(df['Orient_corr'].isin([1,0]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

In [ ]:
Counter(df[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)]['Shape'].tolist())

In [31]:
#get orientation types
indices = df.index[(df['Orient_corr'].isin([1]))&(df['Shape_corr'] == 1)].tolist()

In [13]:
# json_path = home +  '/llama_aug_synth_function_output_v2.json'
json_path = home +  '/neural_builder_lvl1_synth_function_output.json'

with open(json_path, 'r') as j:
    jfile = json.load(j)
    samples = jfile

In [14]:
orient_dict = defaultdict(list)

for sample in samples:
    shape = sample['shape'] 
    if sample['index'] in indices:
        orient_dict[shape].append(sample['orient'])
        

In [ ]:
Counter(orient_dict['rectangle'])

In [ ]:
Counter(orient_dict['diamond'])

#### for each incorrect square, get the instruction size, then check if square

In [ ]:
import functions as fn

In [ ]:
rec_squares = [['index', 'inst_size', 'rec_size']]

In [ ]:
#get squares that are wrong because they are rectangles -- maybe try "is rectangle" in these cases
indices = df.index[df['Shape'].isin(['square'])& df['Shape_corr'].isin([0])].tolist()

In [ ]:
len(indices)

In [ ]:
indices[:3]

In [ ]:
# json_path = home + '/llama_synth_function_output.json'

# with open(json_path, 'r') as j:
#     jfile = json.load(j)
#     samples = jfile

In [ ]:
for ind in indices:
    shape = samples[ind]['shape']
    inst = samples[ind]['size']
    seq = samples[ind]['net_seq']
    rec_size = None
    is_rectu = fn.is_rectangle_unfilled(seq)
    is_rect = fn.is_rectangle(seq)
    if is_rect:
        rec_size = is_rect[1]
    elif is_rectu:
        rect_size = is_rectu[1]
    rec_squares.append([ind, inst, rec_size])



In [ ]:
rec_squares

In [ ]:
len(rec_squares)

In [ ]:
import csv
square_path = home + '/rec_squares.csv'

In [ ]:
with open(square_path, 'w', newline='') as cfile:
    writer = csv.writer(cfile)
    writer.writerows(rec_squares)

In [ ]:
indices = df.index[(df['Shape'].isin(['square']))&(df['Shape_corr'] == 1)].tolist()

In [ ]:
indices